# 기사 크롤링(2021.03.17 ~ 2021.03.31 사이 10페이지)

In [22]:
# https://chromedriver.chromium.org/downloads 버전에 맞는 크롬 드라이버 설치

In [23]:
# !pip install selenium -q

In [24]:
# !pip install webdriver-manager -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [147]:
# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_dt,end_dt, start_dt1,_end_dt1,start_pg ,end_pg):
    if start_pg == end_pg:
        "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&sort=0&photo=0&field=0&pd=3&ds=" + start_dt +"&de=" + end_dt+"&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from"+ start_dt1+"to"+end_dt1+",a:all&start="+str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&sort=0&photo=0&field=0&pd=3&ds=" + start_dt +"&de=" + end_dt+"&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from"+ start_dt1+"to"+end_dt1+",a:all&start="+str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls


################### 뉴스크롤링 시작###################

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ", page2, "페이지")
# 검색 시작할 페이지 입력
start_dt= input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.07.01:")  # ex)2022.07.01
print("\n크롤링할 시작 날짜 : ", start_dt, "페이지")
# 검색 시작할 페이지 입력
start_dt1 = input("\n크롤링할 시작 날짜를 입력해주세요. ex)20220701:")  # ex) 20220701
print("\n크롤링할 시작 날짜 : ", start_dt1, "페이지")
# 검색 시작할 페이지 입력
end_dt = input("\n크롤링할 마지막 날짜를 입력해주세요. ex)2022.07.01:")  # ex) 2022.07.01
print("\n크롤링할 마지막 날짜 : ", end_dt, "페이지")
# 검색 시작할 페이지 입력
end_dt1 = input("\n크롤링할 마지막 날짜를 입력해주세요. ex)20220701:")  # ex) 20220701
print("\n크롤링할 마지막 날짜 : ", end_dt1, "페이지")



# naver url 생성
search_urls = makeUrl(search,start_dt, end_dt, start_dt1, end_dt1, page, page2)

## selenium으로 navernews만 뽑아오기##

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(2)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # 뉴스 본문 가져오기

    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    contents.append(content)
    

# 데이터프레임으로 정리(titles,url,contents)
import pandas as pd
import numpy as np

news_df = pd.DataFrame({'title': titles, 'link': naver_urls, 'content': contents})


검색할 키워드를 입력해주세요:삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):10

크롤링할 종료 페이지:  10 페이지

크롤링할 시작 날짜를 입력해주세요. ex)2022.07.01:2021.03.17

크롤링할 시작 날짜 :  2021.03.17 페이지

크롤링할 시작 날짜를 입력해주세요. ex)20220701:20210317

크롤링할 시작 날짜 :  20210317 페이지

크롤링할 마지막 날짜를 입력해주세요. ex)2022.07.01:2021.03.31

크롤링할 마지막 날짜 :  2021.03.31 페이지

크롤링할 마지막 날짜를 입력해주세요. ex)20220701:20210331

크롤링할 마지막 날짜 :  20210331 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&sort=0&photo=0&field=0&pd=3&ds=2021.03.17&de=2021.03.31&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20210317to20210331,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자&sort=0&photo=0&field=0&pd=3&ds=2021.03.17&de=2021.03.31&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20210317to20210331,a:all&start=11', 'https://search.naver.com/search.nave

https://www.kmib.co.kr/news/index.asp
https://n.news.naver.com/mnews/article/005/0001425741?sid=101
http://www.hg-times.com/
http://www.news2day.co.kr/
http://www.queen.co.kr/
https://www.getnews.co.kr/
https://www.ajunews.com/
https://biz.sbs.co.kr/
https://n.news.naver.com/mnews/article/374/0000238995?sid=101
https://moneys.mt.co.kr/
https://n.news.naver.com/mnews/article/417/0000674846?sid=105
https://biz.chosun.com/
https://n.news.naver.com/mnews/article/366/0000692757?sid=102
http://www.econovill.com/
https://www.asiae.co.kr/
https://n.news.naver.com/mnews/article/277/0004875986?sid=101
http://www.updownnews.co.kr/
https://www.dnews.co.kr/
['https://n.news.naver.com/mnews/article/001/0012295869?sid=105', 'https://n.news.naver.com/mnews/article/003/0010421765?sid=101', 'https://n.news.naver.com/mnews/article/277/0004875367?sid=101', 'https://n.news.naver.com/mnews/article/215/0000947373?sid=101', 'https://n.news.naver.com/mnews/article/366/0000693445?sid=105', 'https://n.news.naver

In [149]:
news_df.info() # 데이터 탐색

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    52 non-null     object
 1   link     52 non-null     object
 2   content  52 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [150]:
news_df.isna().sum() # 결측치 확인

title      0
link       0
content    0
dtype: int64

# 기사 날짜 데이터 추출(2021.03.17 ~ 2021.03.31 사이 10페이지)

In [148]:
# 크롤링 결과 저장 리스트 선언
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]
result={}

# 날짜 정제화 함수
def date_cleansing(test):
    try:
        # 지난 뉴스
        pattern = '\d+.(\d+).(\d+).'  # 정규 표현식 활용
        r = re.compile(pattern)
        match = r.search(test).group(0) 
        date.append(match)
        
    except AttributeError:
        # 최근 뉴스
        pattern = '\w* (\d\w*)'     # 정규 표현식 활용
        r = re.compile(pattern)
        match = r.search(test).group(1)
        date.append(match)

# 내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip() #뒤에 필요없는 부분 제거 
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 페이지를 넘기면 1->11->
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + query + "&sort="+sort+"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&cluster_rank=168&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from" + s_from + "to" + e_to + ",a:all&start=" + str(page)
        
        response = requests.get(url)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text) #신문사
        
        #날짜 추출 
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            # 1면 3단 같은 위치 제거
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)
        
        #본문요약본
        contents_lists = soup.select('.news_dsc')
        for contents_list in contents_lists:
            contents_cleansing(contents_list) #본문요약 정제화
        

        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        print(page)
        
        df = pd.DataFrame(result)  #df로 변환
        page += 10
    
    # 새로 만들 파일이름 지정
    df.to_excel('news_date.xlsx',sheet_name = 'sheet1')

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    maxpage = input("최대 크롤링할 페이지 수 입력하시오 : ")  
    query = input("검색어 입력 : ")  
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1 오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(ex : 2021.03.17):") 
    e_date = input("끝날짜 입력(ex : 2021.03.31):")   
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()



입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오 : 10
검색어 입력 : 삼성전자
뉴스 검색 방식 입력(관련도순=0  최신순=1 오래된순=2): 0
시작날짜 입력(ex : 2021.03.17):2021.03.17
끝날짜 입력(ex : 2021.03.31):2021.03.31
1
11
21
31
41
51
61
71
81
91


In [151]:
news_date = pd.read_excel('news_date.xlsx')

In [152]:
news_date['title']="["+news_date['title']+"]" # 조인을 위해 양쪽에 [] 추가

In [180]:
news_df

,title,content
0,"[삼성전자, 갤럭시 S10 친환경 포장재로 미국 환경청서 상받아]",[\n'2020 지속가능 소재 관리상'…재활용 확대로 7년 연속 우수기업 지정\n\...
1,"[삼성전자, ‘비스포크 인덕션’ 신제품 내일 출시]","[\n코드식 단일 화구 기준, 국내 최고 수준 화력 3300W 구현\n\n\n\n[..."
2,"[삼성전자 美 오스틴 공장, 셧다운 6주만에 가동 정상화]",[\n\t\t\t[아시아경제 우수연 기자]미국 남부 지역의 기록적인 한파로 가동이 ...
3,"[삼성전자, 내달부터 '국민가전 페스타'…최대 80만원 할인]",[\n4월1일~6월30일 전국 온·오프라인 매장 실시총 12개 품목·64개 모델 할...
4,"[삼성전자, 비스포크 인덕션 신제품 출시]",[\n\t\t\t삼성전자는 31일 ‘삼성 비스포크 인덕션'을 내달 1일 출시한다고 ...
5,[롯데도 상반기 수시채용 전환… 5대그룹중 삼성만 공채 유지],[\n[달라지는 채용]롯데 “경영환경 급변… 필요할때 채용”계열사 수요 맞춰 채용방...
6,[대법 “공기청정기 바이러스 99%제거는 과장광고”..삼성전자 사실상 패소],[\n\n\n\n\n [파이낸셜뉴스] '바이러스 제거 99.99%' 등 실험실 결과...
7,"[삼성 QLED 8K TV, 업계 최초 Wi-Fi 6E 인증 획득]",[\n빠르고 안정적인 초고속 무선 인터넷 제공\n\n\n\n[서울=뉴시스] 삼성전자...
8,[삼성·현대차 등 6개 그룹 금융 계열사 관리감독 강화],[\n'금융복합기업집단 감독' 세부안 확정6월 30일 시행비금융 계열사 리스크 고려...
9,[샤오미 100만원대 폴더블폰?...삼성은 다 계획이 있다],[\n\t\t\t[머니투데이 박효주 기자] [[킥킥IT!] 치고나가는 IT뉴스]\...


In [181]:
news_date

,title,date
0,"[삼성전자, 갤럭시 S10 친환경 포장재로 미국 환경청서 상받아]",2021.03.31.
1,"[삼성전자, ‘비스포크 인덕션’ 신제품 내일 출시]",2021.03.31.
2,"[삼성전자 美 오스틴 공장, 셧다운 6주만에 가동 정상화]",2021.03.30.
3,"[삼성전자, 내달부터 '국민가전 페스타'…최대 80만원 할인]",2021.03.30.
4,"[삼성전자, 비스포크 인덕션 신제품 출시]",2021.03.31.
...,...,...
95,"[[특징주]갤럭시아머니, 삼성전자도 '제2의 비트코인' NFT 눈독…시장선점 기대]",2021.03.31.
96,"[삼성전자 ""국민가전 구매하고 풍성한 혜택 누리세요""]",2021.03.30.
97,"[삼성전자, 차기 갤럭시워치에 안드로이드 탑재할까]",2021.03.31.
98,"[삼성 네오 QLED 8K, 와이파이 6E 인증…""TV 업계 최초""]",2021.03.30.


# 데이터 프레임 조인

In [170]:
news_df = news_df[['title','content']]
news_date = news_date[['title','date']]

In [171]:
news = pd.merge(news_df, news_date, left_on='title', right_on='title', how='inner') # 이너 조인

In [199]:
news = news.drop_duplicates() # 중복되는 기사 삭제

# 일일 기사 추출

In [208]:
n_0331 = news[news['date'] == '2021.03.31.'] # 3월 31일 기사만 추출
n_0331 = n_0331.reset_index(drop=True,inplace=False) # 자연어 처리를 위한 인덱싱 초기화

In [209]:
n_0330 = news[news['date'] == '2021.03.30.'] # 3월 30일 기사만 추출
n_0330 = n_0330.reset_index(drop=True,inplace=False)

In [210]:
n_0329 = news[news['date'] == '2021.03.29.'] # 3월 29일 기사만 추출
n_0329 = n_0329.reset_index(drop=True,inplace=False)

# 불용어 사전

In [99]:
korean_stopwords_path = "korean_stopwords.txt"

In [100]:
with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

# 자연어 처리(일자별 빈도 상위 40개 추출)

In [ ]:
#!git clone https://github.com/lovit/textrank.git
#!pip install pororo
#!pip install konlpy

In [211]:
import konlpy
from konlpy.tag import Okt
from collections import Counter

In [212]:
sum = ""
ranked_tags = ""
for i in range(len(n_0331)):
    a = n_0331['content'][i] 
    sum += a
    nouns_tag = Okt()
    nouns = nouns_tag.nouns(sum)
    count = Counter(nouns)
    remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
    remove_char_counter = Counter({x: remove_char_counter[x] for x in count if x not in stopwords})
    r_0331 = remove_char_counter.most_common(40)

# 동일한 방법으로 날짜별 빈도 산출

In [213]:
sum = ""
ranked_tags = ""
for i in range(len(n_0330)):
    a = n_0330['content'][i] 
    sum += a
    nouns_tag = Okt()
    nouns = nouns_tag.nouns(sum)
    count = Counter(nouns)
    remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
    remove_char_counter = Counter({x: remove_char_counter[x] for x in count if x not in stopwords})
    r_0330 = remove_char_counter.most_common(40)

In [214]:
sum = ""
ranked_tags = ""
for i in range(len(n_0329)):
    a = n_0329['content'][i] 
    sum += a
    nouns_tag = Okt()
    nouns = nouns_tag.nouns(sum)
    count = Counter(nouns)
    remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
    remove_char_counter = Counter({x: remove_char_counter[x] for x in count if x not in stopwords})
    r_0329 = remove_char_counter.most_common(40)

# 빈도 데이터프레임 생성

In [236]:
data_0331 = pd.DataFrame(r_0331, columns = ['단어','20210331'])
data_0330 = pd.DataFrame(r_0330, columns = ['단어','20210330'])
data_0329 = pd.DataFrame(r_0329, columns = ['단어','20210329'])

In [248]:
data = pd.merge(data_0330, data_0331, on = '단어', how = 'outer')

In [250]:
data = pd.merge(data,data_0329, on = '단어', how = 'outer')

In [254]:
data['mean'] = data.mean(axis = 'columns') # 기간 내 언급 평균치

In [255]:
data

,단어,20210330,20210331,20210329,mean
0,삼성,130.0,100.0,3.0,77.666667
1,공채,96.0,NaN,NaN,96.000000
2,그룹,84.0,NaN,NaN,84.000000
3,채용,80.0,NaN,NaN,80.000000
4,제품,63.0,50.0,2.0,38.333333
...,...,...,...,...,...
96,확보,NaN,NaN,2.0,2.000000
97,비보,NaN,NaN,2.0,2.000000
98,최근,NaN,NaN,2.0,2.000000
99,사태,NaN,NaN,2.0,2.000000


## 